In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess

In [2]:
# variables

learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "cora"

tf.random.set_seed(seed=seed)  # make weight initialization reproducible


In [3]:
# Load data
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


c:\Users\gsevr\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

In [9]:
# model
model = GCN(n_labels=dataset.n_labels)
model.compile(
    optimizer=Adam(learning_rate),
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)


In [1]:


# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Pre-processing node features


c:\Users\gsevr\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1.9540 - acc: 0.0929 - val_loss: 1.9497 - val_acc: 0.3580
Epoch 2/200
1/1 [==============================] - 0s 78ms/step - loss: 1.9479 - acc: 0.4143 - val_loss: 1.9460 - val_acc: 0.4640
Epoch 3/200
1/1 [==============================] - 0s 70ms/step - loss: 1.9424 - acc: 0.4929 - val_loss: 1.9422 - val_acc: 0.5320
Epoch 4/200
1/1 [==============================] - 0s 76ms/step - loss: 1.9366 - acc: 0.5857 - val_loss: 1.9382 - val_acc: 0.6020
Epoch 5/200
1/1 [==============================] - 0s 81ms/step - loss: 1.9291 - acc: 0.6571 - val_loss: 1.9341 - val_acc: 0.6380
Epoch 6/200
1/1 [==============================] - 0s 77ms/step - loss: 1.9227 - acc: 0.6786 - val_loss: 1.9298 - val_acc: 0.6640
Epoch 7/200
1/1 [==============================] - 0s 76ms/step - loss: 1.9117 - acc: 0.7286 - val_loss: 1.9254 - val_acc: 0.6880
Epoch 8/200
1/1 [==============================] - 0s 77ms/step - loss: 1.9092 - acc: 0.7429